<a href="https://colab.research.google.com/github/alvaroferrerrizzo/Employee-Churn-Predictive-Model/blob/master/Reby/alvaro_random/kernel_api_cardpayins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json
import pandas as pd
import requests
import time
import os
import datetime
from google.cloud import bigquery
import re

In [1]:
cos = ['reby','oiz','avant','tucycle','iberscot','reby_spain','reby_italy']
cos_ids = ['acc_3rjqcmjjct5qey8tstf1','acc_3rjqcudcqm5817xsldx1',
           'acc_3rjqcqpfc144km6bdch1','acc_3rjqcs9fnxk3la3zp5yh',
           'acc_3rjqctd816yhqq7ufj81','acc_3su61k4l3n61dfmsgrah',
           'acc_3su61kkje19lqp9a728h']

In [ ]:
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="reby-cloud.json"

In [ ]:
def send_to_bigquery(df,dataset,table):
	#client = bigquery.Client(project = 'reby-cloud')
	table_id = 'reby-cloud.'+dataset+'.'+table
	print('table_id: {}'.format(table_id))
	#print(df)
	job_config = bigquery.LoadJobConfig()
	#job_config.schema_autodetect = True
	job_config.write_disposition = 'WRITE_TRUNCATE'
	job_config.create_disposition = 'CREATE_IF_NEEDED'
	job = client.load_table_from_dataframe(df, table_id,job_config=job_config)
	# Wait for the load job to complete.
	result = job.result()
	#print(result)
	return 'dataframe appended'

In [ ]:
"""
client = bigquery.Client(project = 'reby-cloud')
#get the dataframe with the last update from bigquery
df_last_update = client.query('''
    select
    company as relevant_company,
    max(ts) as max_ts
	from (

	select 
		company,
		timestamp(regexp_extract(created_at_main, r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}.\d{4}")) as ts
	from `reby-cloud.kernel_eu.bank_payout_list`

	union all 

	select 
		company,
		timestamp(regexp_extract(created_at_main, r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}.\d{4}")) as ts
	from `reby-cloud.kernel_eu.transfer_list`

	union all 

	select 
		company,
		timestamp(regexp_extract(created_at_main, r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}.\d{4}")) as ts
	from `reby-cloud.kernel_eu.bank_payout_list`
	)
	group by 1
''' ).to_dataframe()

df_last_update['max_ts'] = df_last_update['max_ts'].apply(lambda x: x.replace(tzinfo=None))
"""

In [37]:
response_json['data'][0]

{'account_id': 'acc_3su61kkje19lqp9a728h',
 'authentication_redirect_url': 'https://api.reby.co/v2/webhooks/kernelpay/paymentdone/c_3qteetfhxjshx4j54111',
 'authentication_type': 'three_d_secure_2',
 'authentication_url': '',
 'calculated_statement_descriptor': 'EP* Reby',
 'card_id': 'crd_3t3p4hzl5xbpze4hyzkh',
 'card_info': {'bank': None,
  'brand': 'MasterCard',
  'country': 'GB',
  'expiration_month': 5,
  'expiration_year': 2025,
  'fingerprint': 'C3GMH6AI/1ZOoJiO',
  'first6': '535584',
  'funding': 'debit',
  'number': '535584******9541'},
 'configure_offline_card': None,
 'created_at': '2021-07-09T12:50:01.819234304Z',
 'decline_code': None,
 'id': 'cpi_3t931mgktmzju6hykczh',
 'meta': {'_redsys_error_code': '0000',
  'company_id': 'c_3qteetfhxjshx4j54111',
  'top_up_id': 'to_3t931mgj7vzf1tdyf3kh',
  'transaction_id': 'tx_3t931mgj7xhpnmsvglb1',
  'transaction_type': 'pre_ride_charge',
  'user_id': 'usr_3t3p47vfeb4ksvmd7ya1',
  'user_session_id': 'sess_3t4782ypjcsvd56hsy7h'},
 'o

In [31]:
auth_token='live-mob_secretkey_3san8rtxkt5ezlbz34m1_43c759a649130e2512e45d8a071d15c8904b391bd808b5a2'
hed = {'Authorization': 'Bearer ' + auth_token}
final_json = []
url_main = 'https://api.kernelpay.com/v1'

#account_id = 'acc_3rjqcudcqm5817xsldx1'
url_cardpayin_tx = url_main+'/card_payins?limit=10'

print(url_cardpayin_tx)

#url = 'https://api.kernelpay.com/v1/accounts/acc_3rjqcudcqm5817xsldx1/transactions?expand=transfer'
#url_next = 'https://api.kernelpay.com/v1/accounts/acc_3rjqcudcqm5817xsldx1/transactions?expand=transfer'

response = requests.get(url_cardpayin_tx
                        , headers=hed)
#print(response)
#print(response.json())
response_json = response.json()

#response_json['created_at']

https://api.kernelpay.com/v1/card_payins?limit=10


In [32]:
len(response_json)
response_json.keys()

dict_keys(['data', 'has_next', 'cursor_next'])

In [30]:
len(response_json['data'])

10

In [16]:
cursor_next = response_json.get('cursor_next')
has_next = str(response_json.get('has_next'))

In [21]:
print(response_json['has_next'])

True


In [ ]:
i=0
has_next = 'True'
continue_ = 'True'


while has_next == 'True' and continue_ == 'True' and i<30:# and i<=60:
#print( "i counter {}".format(i))
    if i == 0:
        fetch_url = url_account_tx
    elif i>0:
        fetch_url = url_cardpayin_tx+'&cursor='+cursor_next

    response = requests.get(fetch_url, headers=hed)
    response_json = response.json()
    
    for j in range(len(response_json['data'])):

        api_pinged_at = 
        created_at =
        id
        authentication_type
        account_id = None
        card_id = 
        card_info_bank = None
        card_info_brand
        card_info_country
        card_info_funding
        decline_code
        configure_offline_card
        meta_redsys_error_code
        meta_company_id
        meta_top_up_id
        meta_transaction_id
        meta_user_id
        meta_transaction_type

        i = i + 1
        time.sleep(0.1)

"""
{'account_id': 'acc_3su61kkje19lqp9a728h',
 'authentication_redirect_url': 'https://api.reby.co/v2/webhooks/kernelpay/paymentdone/c_3qteetfhxjshx4j54111',
 'authentication_type': 'three_d_secure_2',
 'authentication_url': '',
 'calculated_statement_descriptor': 'EP* Reby',
 'card_id': 'crd_3t3p4hzl5xbpze4hyzkh',
 'card_info': {'bank': None,
  'brand': 'MasterCard',
  'country': 'GB',
  'expiration_month': 5,
  'expiration_year': 2025,
  'fingerprint': 'C3GMH6AI/1ZOoJiO',
  'first6': '535584',
  'funding': 'debit',
  'number': '535584******9541'},
 'configure_offline_card': None,
 'created_at': '2021-07-09T12:50:01.819234304Z',
 'decline_code': None,
 'id': 'cpi_3t931mgktmzju6hykczh',
 'meta': {'_redsys_error_code': '0000',
  'company_id': 'c_3qteetfhxjshx4j54111',
  'top_up_id': 'to_3t931mgj7vzf1tdyf3kh',
  'transaction_id': 'tx_3t931mgj7xhpnmsvglb1',
  'transaction_type': 'pre_ride_charge',
  'user_id': 'usr_3t3p47vfeb4ksvmd7ya1',
  'user_session_id': 'sess_3t4782ypjcsvd56hsy7h'},
 'offline': True,
 'project_id': 'prj_3rhmr1vp8xcnxvfktus1',
 'refunded_value': {'amount': 0, 'currency': 'EUR'},
 'sca_card_id': None,
 'state': 'authorized',
 'transaction_id': None,
 'updated_at': '2021-07-09T12:50:03.077334Z',
 'value': {'amount': 50, 'currency': 'EUR'}}
"""

In [ ]:
#Bearer live_secretkey_3m9khmufqsgct1llddu1_13383aeb28127115180a92bac880957cc6faa71936b44529

auth_token='live-mob_secretkey_3san8rtxkt5ezlbz34m1_43c759a649130e2512e45d8a071d15c8904b391bd808b5a2'
hed = {'Authorization': 'Bearer ' + auth_token}

url_main = 'https://api.kernelpay.com/v1'

transfer_list = []
cardpayin_list = []
bankpayout_list = []
error_list = []

def fetch_kernel_data(event, context):
	#-------
	#fetch data from kernel and upload it to bigquery
	#-------
	for company,account_id in zip(cos,cos_ids):
		#Get the last timestamp available for that company
		mask = df_last_update['relevant_company'] == company
		company_data = df_last_update[mask]
		last_timestamp_company_bigquery = company_data.max_ts
		#print(f'last timestamp bigquery {last_timestamp_company_bigquery} for {company}')
		#------

		print(company)
		print(account_id)
	
		#set core url
		url_account_tx = url_main+'/accounts/'+account_id+'/transactions?expand=transfer,card_payin&limit=50'
		i=0
		has_next = 'True'
		continue_ = 'True'
		
		while has_next == 'True' and continue_ == 'True':# and i<=60:
		
		#print( "i counter {}".format(i))
			if i == 0:
				fetch_url = url_account_tx
			elif i>0:
				fetch_url = url_account_tx+'&cursor='+cursor_next

			response = requests.get(fetch_url, headers=hed)
			response_json = response.json()
			
			for j in range(len(response_json['data'])):
				
				#reset variables
				#-----
				id = None
				amount = None
				currency = None
				type_main = None
				created_at = None
				destination_account_id = None
				destination_transaction_id = None
				transfer_id = None
				source_account_id = None
				source_transaction_id = None
				type_meta = None
				card_payin_id = None
				card_payin_refund_id = None
				refunded_value = None
				transaction_id_meta = None
				#-----
				#print("type - {}".format(response_json['data'][0]['type']))

				try:
					id = response_json['data'][j]['id']
					amount = response_json['data'][j]['value']['amount']
					currency = response_json['data'][j]['value']['currency']
					type_main = response_json['data'][j]['type']
					created_at_main = response_json['data'][j]['created_at']

					if type_main == 'transfer_destination' or type_main == 'transfer_source':
						created_at = response_json['data'][j]['transfer']['created_at']
						destination_account_id = response_json['data'][j]['transfer']['destination_account_id']
						destination_transaction_id = response_json['data'][j]['transfer']['destination_transaction_id']
						transfer_id = response_json['data'][j]['transfer']['id']
						source_account_id = response_json['data'][j]['transfer']['source_account_id']
						source_transaction_id = response_json['data'][j]['transfer']['source_transaction_id']
						try:
							type_meta = response_json['data'][j]['transfer']['meta']['transaction_type']
						except:
							type_meta = 'not_available'
						try:
							backend_account_id = response_json['data'][j]['transfer']['meta']['backend_account_id']
						except:
							backend_account_id = None
							
						transfer_list.append([company,id,created_at,created_at_main,destination_account_id
										,destination_transaction_id, transfer_id, source_account_id, 
										source_transaction_id, amount, currency, type_main, type_meta, backend_account_id])
						#i = i + 1
						#print("success")

					elif type_main in ['card_payin','card_payin_refund','card_payin_dispute',
                                    'card_payin_dispute_refund','card_payin_dispute_lost']:
                            
						created_at = response_json['data'][j]['card_payin']['created_at']
						destination_account_id = response_json['data'][j]['account_id']
						card_payin_id = response_json['data'][j]['card_payin_id']
						card_payin_refund_id = response_json['data'][j]['card_payin_refund_id']
						refunded_value = response_json['data'][j]['card_payin']['refunded_value']['amount']
						
						try:
							type_meta = response_json['data'][j]['card_payin']['meta']['transaction_type']
						except:
							type_meta = 'not_available'

						try:
							transaction_id_meta = response_json['data'][j]['card_payin']['meta']['transaction_id']
						except:
							transaction_id_meta = None

						try:
							top_up_id = response_json['data'][j]['card_payin']['meta']['top_up_id']
						except:
							top_up_id = None
						
						#user_id = response_json['data'][0]['card_payin']['meta']['user_id']


						cardpayin_list.append([company,id,created_at,created_at_main,destination_account_id,
												amount, currency, type_main, type_meta, card_payin_id,
												card_payin_refund_id,transaction_id_meta,refunded_value, top_up_id])
						#i = i + 1
						#print("success")

					elif type_main == 'bank_payout':

						bankpayout_list.append([company,id,amount,currency,type_main,created_at_main])
						#i = i + 1

					else:
						id = response_json['data'][j]['id']
						amount = response_json['data'][j]['value']['amount']
						type_main = response_json['data'][j]['type']
						error_list.append([company,id,amount,type_main,'unknown_type'])
						print("here")

				except Exception as e:
					id = response_json['data'][j]['id']
					amount = response_json['data'][j]['value']['amount']
					type_main = response_json['data'][j]['type']
					error_list.append([company,id,amount,type_main,e])
					#print(response_json)
					#i = i + 1
					#print("error")
					print("here2")
					#print(type_main)
					#print(id)

			created_at_str = created_at_main
			timestamp_regex = re.findall(
						r"\d{4}-\d{1,2}-\d{1,2}T\d{1,2}:\d{1,2}:\d{1,2}.\d{1,5}",created_at_str)
			created_at_regex_unique = timestamp_regex[0]
			created_at_formatted = datetime.datetime.strptime(created_at_regex_unique, '%Y-%m-%dT%H:%M:%S.%f')
			

			if company_data.empty:
				continue_ = 'True'

			elif created_at_formatted and created_at_formatted >  last_timestamp_company_bigquery.item():
				#print(f'last created_at in kernel batch: {created_at_formatted}')
				#print(f'last created_at  in bigquery: {last_timestamp_company_bigquery.item()}')
				continue_ = 'True'
			else:
				#print(f'last created_at in kernel batch: {created_at_formatted}')
				#print(f'last created_at in bigquery: {last_timestamp_company_bigquery.item()}')
				continue_ = 'False'

			cursor_next = response_json.get('cursor_next')
			has_next = str(response_json.get('has_next'))
			#print("cursor_next {}".format(cursor_next))
			#print(f'hast next: {has_next}')
			#print(f'continue: {continue_}')
			i = i + 1
			time.sleep(0.1)
	return bankpayout_list, cardpayin_list, transfer_list
  
bankpayout_list, cardpayin_list, transfer_list = fetch_kernel_data('a','b')

In [ ]:
#Bearer live_secretkey_3m9khmufqsgct1llddu1_13383aeb28127115180a92bac880957cc6faa71936b44529

auth_token='live-mob_secretkey_3san8rtxkt5ezlbz34m1_43c759a649130e2512e45d8a071d15c8904b391bd808b5a2'
hed = {'Authorization': 'Bearer ' + auth_token}

url_main = 'https://api.kernelpay.com/v1'

transfer_list = []
cardpayin_list = []
bankpayout_list = []
error_list = []

def fetch_kernel_data(event, context):
	#-------
	#fetch data from kernel and upload it to bigquery
	#-------
	for company,account_id in zip(cos,cos_ids):
		#Get the last timestamp available for that company
		mask = df_last_update['relevant_company'] == company
		company_data = df_last_update[mask]
		last_timestamp_company_bigquery = company_data.max_ts
		#print(f'last timestamp bigquery {last_timestamp_company_bigquery} for {company}')
		#------

		print(company)
		print(account_id)
	
		#set core url
		url_account_tx = url_main+'/accounts/'+account_id+'/transactions?expand=transfer,card_payin&limit=50'
		i=0
		has_next = 'True'
		continue_ = 'True'
		
		while has_next == 'True' and continue_ == 'True':# and i<=60:
		
		#print( "i counter {}".format(i))
			if i == 0:
				fetch_url = url_account_tx
			elif i>0:
				fetch_url = url_account_tx+'&cursor='+cursor_next

			response = requests.get(fetch_url, headers=hed)
			response_json = response.json()
			
			for j in range(len(response_json['data'])):
				
				#reset variables
				#-----
				id = None
				amount = None
				currency = None
				type_main = None
				created_at = None
				destination_account_id = None
				destination_transaction_id = None
				transfer_id = None
				source_account_id = None
				source_transaction_id = None
				type_meta = None
				card_payin_id = None
				card_payin_refund_id = None
				refunded_value = None
				transaction_id_meta = None
				#-----
				#print("type - {}".format(response_json['data'][0]['type']))

				try:
					id = response_json['data'][j]['id']
					amount = response_json['data'][j]['value']['amount']
					currency = response_json['data'][j]['value']['currency']
					type_main = response_json['data'][j]['type']
					created_at_main = response_json['data'][j]['created_at']

					if type_main == 'transfer_destination' or type_main == 'transfer_source':
						created_at = response_json['data'][j]['transfer']['created_at']
						destination_account_id = response_json['data'][j]['transfer']['destination_account_id']
						destination_transaction_id = response_json['data'][j]['transfer']['destination_transaction_id']
						transfer_id = response_json['data'][j]['transfer']['id']
						source_account_id = response_json['data'][j]['transfer']['source_account_id']
						source_transaction_id = response_json['data'][j]['transfer']['source_transaction_id']
						try:
							type_meta = response_json['data'][j]['transfer']['meta']['transaction_type']
						except:
							type_meta = 'not_available'
						try:
							backend_account_id = response_json['data'][j]['transfer']['meta']['backend_account_id']
						except:
							backend_account_id = None
							
						transfer_list.append([company,id,created_at,created_at_main,destination_account_id
										,destination_transaction_id, transfer_id, source_account_id, 
										source_transaction_id, amount, currency, type_main, type_meta, backend_account_id])
						#i = i + 1
						#print("success")

					elif type_main in ['card_payin','card_payin_refund','card_payin_dispute',
                                    'card_payin_dispute_refund','card_payin_dispute_lost']:
                            
						created_at = response_json['data'][j]['card_payin']['created_at']
						destination_account_id = response_json['data'][j]['account_id']
						card_payin_id = response_json['data'][j]['card_payin_id']
						card_payin_refund_id = response_json['data'][j]['card_payin_refund_id']
						refunded_value = response_json['data'][j]['card_payin']['refunded_value']['amount']
						
						try:
							type_meta = response_json['data'][j]['card_payin']['meta']['transaction_type']
						except:
							type_meta = 'not_available'

						try:
							transaction_id_meta = response_json['data'][j]['card_payin']['meta']['transaction_id']
						except:
							transaction_id_meta = None

						try:
							top_up_id = response_json['data'][j]['card_payin']['meta']['top_up_id']
						except:
							top_up_id = None
						
						#user_id = response_json['data'][0]['card_payin']['meta']['user_id']


						cardpayin_list.append([company,id,created_at,created_at_main,destination_account_id,
												amount, currency, type_main, type_meta, card_payin_id,
												card_payin_refund_id,transaction_id_meta,refunded_value, top_up_id])
						#i = i + 1
						#print("success")

					elif type_main == 'bank_payout':

						bankpayout_list.append([company,id,amount,currency,type_main,created_at_main])
						#i = i + 1

					else:
						id = response_json['data'][j]['id']
						amount = response_json['data'][j]['value']['amount']
						type_main = response_json['data'][j]['type']
						error_list.append([company,id,amount,type_main,'unknown_type'])
						print("here")

				except Exception as e:
					id = response_json['data'][j]['id']
					amount = response_json['data'][j]['value']['amount']
					type_main = response_json['data'][j]['type']
					error_list.append([company,id,amount,type_main,e])
					#print(response_json)
					#i = i + 1
					#print("error")
					print("here2")
					#print(type_main)
					#print(id)

			created_at_str = created_at_main
			timestamp_regex = re.findall(
						r"\d{4}-\d{1,2}-\d{1,2}T\d{1,2}:\d{1,2}:\d{1,2}.\d{1,5}",created_at_str)
			created_at_regex_unique = timestamp_regex[0]
			created_at_formatted = datetime.datetime.strptime(created_at_regex_unique, '%Y-%m-%dT%H:%M:%S.%f')
			

			if company_data.empty:
				continue_ = 'True'

			elif created_at_formatted and created_at_formatted >  last_timestamp_company_bigquery.item():
				#print(f'last created_at in kernel batch: {created_at_formatted}')
				#print(f'last created_at  in bigquery: {last_timestamp_company_bigquery.item()}')
				continue_ = 'True'
			else:
				#print(f'last created_at in kernel batch: {created_at_formatted}')
				#print(f'last created_at in bigquery: {last_timestamp_company_bigquery.item()}')
				continue_ = 'False'

			cursor_next = response_json.get('cursor_next')
			has_next = str(response_json.get('has_next'))
			#print("cursor_next {}".format(cursor_next))
			#print(f'hast next: {has_next}')
			#print(f'continue: {continue_}')
			i = i + 1
			time.sleep(0.1)
	return bankpayout_list, cardpayin_list, transfer_list
  
bankpayout_list, cardpayin_list, transfer_list = fetch_kernel_data('a','b')

last timestamp bigquery 0   2021-06-06 06:01:04.867600
Name: max_ts, dtype: datetime64[ns] for reby
reby
acc_3rjqcmjjct5qey8tstf1
last created_at in kernel batch: 2021-06-06 06:01:04.867610
last created_at  in bigquery: 2021-06-06 06:01:04.867600
last created_at in bigquery: 2021-06-06 06:01:04.867600
last timestamp bigquery 1   2021-06-06 22:28:31.094200
Name: max_ts, dtype: datetime64[ns] for oiz
oiz
acc_3rjqcudcqm5817xsldx1
last created_at in bigquery: 2021-06-06 22:28:31.094200
last timestamp bigquery 2   2021-06-06 19:19:29.408200
Name: max_ts, dtype: datetime64[ns] for avant
avant
acc_3rjqcqpfc144km6bdch1
last created_at in bigquery: 2021-06-06 19:19:29.408200
last timestamp bigquery 3   2021-06-06 16:15:48.924100
Name: max_ts, dtype: datetime64[ns] for tucycle
tucycle
acc_3rjqcs9fnxk3la3zp5yh
last created_at in bigquery: 2021-06-06 16:15:48.924100
last timestamp bigquery 4   2021-06-06 22:32:15.514400
Name: max_ts, dtype: datetime64[ns] for iberscot
iberscot
acc_3rjqctd816yhqq7u

In [ ]:
transfer_list_df = pd.DataFrame(transfer_list)
if len(transfer_list) >  0:
    transfer_list_df.columns = ['company','id','created_at','created_at_main','destination_account_id'
                            ,'destination_transaction_id', 'transfer_id', 'source_account_id', 
                            'source_transaction_id', 'amount', 'currency', 'type_main', 'type_meta','backend_account_id']
    

cardpayin_list_df = pd.DataFrame(cardpayin_list)
if len(cardpayin_list) >  0:
    cardpayin_list_df.columns = ['company','id','created_at','created_at_main','destination_account_id',
                                          'amount', 'currency', 'type_main', 'type_meta', 'card_payin_id',
                                 'card_payin_refund_id','transaction_id_meta','refunded_value','top_up_id']

bankpayout_list_df = pd.DataFrame(bankpayout_list)
if len(bankpayout_list) >  0:
    bankpayout_list_df.columns = ['company','id','amount','currency','type_main','created_at_main']

error_list_df = pd.DataFrame(error_list)
if len(error_list) >  0:
    error_list_df.columns = ['company','error_ids','amount','type','reason']


In [ ]:
error_list_df.head(20)

""


In [ ]:
#transfer_list_df[transfer_list_df['type_main']=='card_payin_dispute_lost']
cardpayin_list_df[cardpayin_list_df['type_main']=='card_payin_dispute_lost']

,company,id,created_at,created_at_main,destination_account_id,amount,currency,type_main,type_meta,card_payin_id,card_payin_refund_id,transaction_id_meta,refunded_value,top_up_id
111,avant,tx_3t3ngj4r4srqbys4h8y1,2021-03-18T13:34:16.968658944Z,2021-06-04T09:42:22.077370368Z,acc_3rjqcqpfc144km6bdch1,-6900,EUR,card_payin_dispute_lost,fee,cpi_3spq7b137dtln5q4spc1,None,tx_3spq7azvfalstufvmc1h,0,to_3spq7azv87k7mn5g93lh
112,avant,tx_3t3ngj4qc1hs77hlqql1,2021-03-18T13:35:02.179913728Z,2021-06-04T09:42:22.06439424Z,acc_3rjqcqpfc144km6bdch1,-6800,EUR,card_payin_dispute_lost,fee,cpi_3spq7dma11411zle1thh,None,tx_3spq7dm8y7yt9727cpuh,0,to_3spq7dm8pnt3pjqv9tr1
113,avant,tx_3t3ngj4pank1shqh7fn1,2021-03-18T13:35:39.752751104Z,2021-06-04T09:42:22.046896128Z,acc_3rjqcqpfc144km6bdch1,-5000,EUR,card_payin_dispute_lost,fee,cpi_3spq7ft9hkrvn7zgk2m1,None,tx_3spq7ft3l5sneptsekeh,0,to_3spq7ft3c5zmjxkvdpuh
114,avant,tx_3t3ngj4nman9tkgtv24h,2021-03-16T10:41:16.593168384Z,2021-06-04T09:42:22.035689472Z,acc_3rjqcqpfc144km6bdch1,-6900,EUR,card_payin_dispute_lost,fee,cpi_3spds4h1zptmdkhhvl3h,None,tx_3spds4gymdzfxnbny1e1,0,to_3spds4gydjuza5xc7sqh
115,avant,tx_3t3ngj4mmgkq5yrcvzjh,2021-03-16T10:41:35.526780928Z,2021-06-04T09:42:22.018977792Z,acc_3rjqcqpfc144km6bdch1,-6800,EUR,card_payin_dispute_lost,fee,cpi_3spds5lahqh6dlhpxzv1,None,tx_3spds5l1bbfgpk42787h,0,to_3spds5l14yqatmlzkp71
116,avant,tx_3t3ngj4kaar3lhu5431h,2021-03-16T10:41:53.1741184Z,2021-06-04T09:42:21.979590656Z,acc_3rjqcqpfc144km6bdch1,-5000,EUR,card_payin_dispute_lost,fee,cpi_3spds6m6dbqg8l833ll1,None,tx_3spds6k86fhmatyvpgrh,0,to_3spds6k7zasbhgj2nr7h


In [ ]:
transfer_list_df.to_csv('transfer_list.csv')
cardpayin_list_df.to_csv('card_payin_list.csv')

In [ ]:
#print(json.dumps(response_json, indent=2))

In [ ]:
len(response_json['data'])

10

In [ ]:
auth_token='live-mob_secretkey_3san8rtxkt5ezlbz34m1_43c759a649130e2512e45d8a071d15c8904b391bd808b5a2'
hed = {'Authorization': 'Bearer ' + auth_token}
final_json = []
url_main = 'https://api.kernelpay.com/v1'

account_id = 'acc_3rjqcudcqm5817xsldx1'
url_account_tx = url_main+'/accounts/'+account_id+'/transactions?expand=transfer,card_payin&limit=10'

transaction_id = 'tx_3rn9ejt594fab8b5cnyh'
url_tx  = url_main+'/transactions/'+transaction_id+'?expand=transfer,card_payin&limit=2'

print(url_account_tx)

#url = 'https://api.kernelpay.com/v1/accounts/acc_3rjqcudcqm5817xsldx1/transactions?expand=transfer'
#url_next = 'https://api.kernelpay.com/v1/accounts/acc_3rjqcudcqm5817xsldx1/transactions?expand=transfer'

response = requests.get(url_tx
                        , headers=hed)
#print(response)
#print(response.json())
response_json = response.json()

response_json

https://api.kernelpay.com/v1/accounts/acc_3rjqcudcqm5817xsldx1/transactions?expand=transfer,card_payin&limit=10


{'id': 'tx_3rn9ejt594fab8b5cnyh',
 'project_id': 'prj_3rhmr1vp8xcnxvfktus1',
 'account_id': 'acc_3rjqcmjjct5qey8tstf1',
 'type': 'card_payin',
 'value': {'amount': 2000, 'currency': 'EUR'},
 'balance_after': {'amount': -21, 'currency': 'EUR'},
 'transfer_id': None,
 'bank_payout_id': None,
 'card_payin_id': 'cpi_3rks1xu5bh1bty9vxb7h',
 'card_payin_refund_id': None,
 'card_payin_dispute_id': None,
 'bank_payin_id': None,
 'card_payin': {'id': 'cpi_3rks1xu5bh1bty9vxb7h',
  'project_id': 'prj_3rhmr1vp8xcnxvfktus1',
  'state': 'settled',
  'card_id': None,
  'sca_card_id': None,
  'account_id': 'acc_3rjqcmjjct5qey8tstf1',
  'transaction_id': 'tx_3rn9ejt594fab8b5cnyh',
  'calculated_statement_descriptor': 'KernelPay',
  'authentication_type': 'none',
  'authentication_url': '',
  'authentication_redirect_url': '',
  'value': {'amount': 2000, 'currency': 'EUR'},
  'refunded_value': {'amount': 0, 'currency': 'EUR'},
  'meta': {},
  'offline': None,
  'created_at': '2020-07-27T09:01:25.4314434

In [ ]:
len(response_json['data'])

32

In [ ]:
from google.cloud import bigquery

def send_to_bigquery(df,dataset,table):
     client = bigquery.Client(project = 'reby-cloud')
     table_id = 'reby-cloud.'+dataset+'.'+table
     print('table_id: {}'.format(table_id))
     #print(df)
     job_config = bigquery.LoadJobConfig()
     job_config.schema_autodetect = True
     job_config.write_disposition = 'WRITE_TRUNCATE'
     job_config.create_disposition = 'CREATE_IF_NEEDED'
     job = client.load_table_from_dataframe(df, table_id,job_config=job_config)
     # Wait for the load job to complete.
     result = job.result()
     #print(result)
     return 'dataframe appended'

In [ ]:
send_to_bigquery(transfer_list_df,'kernel_eu','transfer_list')
send_to_bigquery(cardpayin_list_df,'kernel_eu','card_payin_list')
send_to_bigquery(bankpayout_list_df,'kernel_eu','bank_payout_list')

table_id: reby-cloud.kernel_eu.transfer_list


/Users/alvaroferrer/anaconda3/lib/python3.7/site-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning: Unable to determine type of column 'company'.
  warnings.warn(u"Unable to determine type of column '{}'.".format(column))


table_id: reby-cloud.kernel_eu.card_payin_list
table_id: reby-cloud.kernel_eu.bank_payout_list


'dataframe appended'